In [1]:
import torch
import torchvision
import torchtext
import sys
sys.path.append('../')
import config
from config import DATA_PATH
from torchvision.transforms import v2 as transforms
from datamodules import IMDBDataModule, CIFARDataModule, LibriSpeechDataModule, SpeechCommandsDataModule

/Users/timcares/miniforge3/envs/mmrl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
ds = IMDBDataModule(data_path=DATA_PATH,
                    batch_size=32,
                    num_workers=5,
                    num_max_bpe_tokens=512,
                    shuffle=True,
                    drop_last=True,
                    )

In [31]:
ds.prepare_data()

Data already exists. Skip creating it.
Data already exists. Skip creating it.


In [43]:
ds.setup('test')

Load 25000 text examples.


In [33]:
loader = ds.train_dataloader()

In [34]:
loader_iter = iter(loader)

In [39]:
b=next(loader_iter)

In [42]:
b['language_tokens'].shape

torch.Size([32, 512])

In [2]:
ds = EnWik9Dataset(data_path=DATA_PATH,
                   batch_size=32,
                   num_workers=10,
                   num_max_bpe_tokens=512,)

In [ ]:
ds.prepare_data()

In [ ]:
ds.setup()

In [12]:
ds = CIFARDataModule(data_path=DATA_PATH,
                    batch_size=32,
                    num_workers=5,
                    type='cifar10',
                    shuffle=True,
                    drop_last=True,
                    )

In [13]:
ds.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
ds.setup('fit')

In [6]:
loader = ds.train_dataloader()

In [7]:
loader_iter = iter(loader)

In [8]:
b=next(loader_iter)

In [11]:
b['image'].shape

torch.Size([32, 3, 224, 224])

In [2]:
ds = LibriSpeechDataModule(data_path=DATA_PATH,
                    sample_rate=16000,
                    max_sample_size=320000,
                    min_sample_size=32000,
                    precompute_mask_config={
                        'feature_encoder_spec': '[(512, 10, 5)] + [(512, 3, 2)] * 4 + [(512,2,2)] + [(512,2,2)]'
                    },
                    type='train-clean-100',
                    batch_size=32,
                    num_workers=4,
                    shuffle=True,
                    drop_last=True,
                    )

In [3]:
ds.prepare_data()

In [4]:
ds.setup('fit')

In [5]:
loader = ds.train_dataloader()

In [6]:
loader_iter = iter(loader)

In [7]:
b=next(loader_iter)

In [9]:
b

{'id': tensor([13739, 23679, 15584,  2842, 16770, 10733, 24544, 25857, 20602, 27968,
           586,  7061,  1678,  3295, 11294, 25649, 22392, 25551,  1222, 17027,
         16134, 14407, 12839,  8639,  4447, 16349,  3390,  1536, 16905, 10352,
         20260,  9649]),
 'audio': tensor([[ 0.1434,  0.1970,  0.2531,  ...,  0.8795,  0.7710,  0.7199],
         [ 0.5050,  0.7065,  0.8682,  ...,  0.0447,  0.0440,  0.0526],
         [ 2.0687,  2.8521,  1.8890,  ..., -0.0058, -0.0086, -0.0091],
         ...,
         [ 2.6879,  2.8653,  2.8942,  ..., -0.0086, -0.1261, -0.2539],
         [ 0.9282,  0.8093,  0.9436,  ...,  0.9603,  0.8756,  0.7874],
         [ 0.4045, -0.7784, -1.6616,  ...,  0.1505, -0.0950, -0.4152]]),
 'precomputed_mask': tensor([[1., 0., 0.,  ..., 1., 1., 0.],
         [1., 0., 1.,  ..., 1., 1., 1.],
         [1., 0., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 0.,  ..., 1., 0., 1.],
         [1., 1., 1.,  ..., 0., 1., 1.],
         [1., 1., 0.,  ..., 0., 1., 1.]])}

In [2]:
ds = SpeechCommandsDataModule(data_path=DATA_PATH,
                              feature_encoder_spec='[(512, 10, 5)] + [(512, 3, 2)] * 4 + [(512,2,2)] + [(512,2,2)]',
                              batch_size=32,
                              num_workers=5,
                              shuffle=True,
                              drop_last=True,
                              )

In [3]:
ds.prepare_data()

In [4]:
ds.setup('fit')

In [5]:
loader = ds.train_dataloader()

In [6]:
loader_iter = iter(loader)

In [7]:
b=next(loader_iter)

In [8]:
b

{'audio': tensor([[ 9.1553e-05,  1.2207e-04,  1.5259e-04,  ...,  6.1035e-05,
           6.1035e-05,  6.1035e-05],
         [ 2.1057e-03,  3.8757e-03,  5.0659e-03,  ..., -7.0496e-03,
          -6.8665e-03, -5.3101e-03],
         [-5.4932e-04, -7.0190e-04, -4.2725e-04,  ...,  0.0000e+00,
          -9.1553e-05, -1.2207e-04],
         ...,
         [ 3.0518e-05, -9.1553e-05, -9.1553e-05,  ...,  3.0518e-05,
          -3.0518e-05, -1.2207e-04],
         [ 0.0000e+00, -6.1035e-05, -9.1553e-05,  ..., -3.0518e-05,
           3.3569e-04,  6.7139e-04],
         [ 4.2725e-04,  7.6294e-04,  1.1292e-03,  ..., -5.6366e-02,
          -6.3782e-02, -6.4484e-02]]),
 'label': tensor([29, 17,  8, 22, 13,  3,  6, 27, 26,  8, 29, 23, 16,  1,  1,  4, 28, 21,
         14, 22, 19, 16, 20, 20, 10, 30, 10, 13, 12, 10,  4,  8]),
 'id': tensor([79817, 58067, 25032, 74100, 35973, 69012,  1448,  8023, 27665, 25367,
         79131, 75755, 66392, 12484, 12545,   798, 59311, 54927, 32393, 73040,
          3562, 67160, 8